# 6. 학습 관련 기술들

* 이번 장에서는 신경망 학습의 핵심 개념인 가중치 매개변수의 최적값을 탐색하는 <span style='color:blue'>최적화 방법</span>, <span style='color:blue'>가중치 매개변수 초깃값</span>, <span style='color:blue'>하이퍼파라미터 설정 방법</span> 등을 학습한다.
* 오버피팅의 대응책인 가중치 감소와 드롭아웃등의 정규화 방법도 짧게 알아보자.
* 그리고 마지막으로 배치 정규화도 짧게 알아보자.

## 6.1 매개변수 갱신

* 신경망 학습의 목적은 손실함수의 값을 가능한 한 낮추는 매개변수를 찾는 것이다.
* 이는 곧 매개변수의 최적값을 찾는 문제이며, 이러한 문제를 푸는 것을 <span style='color:blue'>최적화(optimization)</span>이라고 한다.
* SGD는 단순하지만, SGD보다 똑똑한 방법도 있다.
* SGD의 단점을 알아본 후 SGD와는 다른 최적화 기법을 살펴보자.

### 6.1.2 확률적 경사 하강법(SGD)

* SGD를 수식으로 나타내면 아래와 같다.<br/>
## $\mathbf{W} \leftarrow {\mathbf{W}} - {\eta{\partial{L}\over\partial{\mathbf{W}}}}$<br/>
* 여기에서 $\mathbf{W}$는 갱신할 가중치 매개변수고 ${\partial{L}\over\partial{\mathbf{W}}}$은 $\mathbf{W}$에 대한 손실함수의 기울기이다.
* $\eta$는 학습률을 의미하는데, 실제로는 0.01이나 0.001과 같은 값을 미리 정해서 사용한다.
* SGD를 구현해보면 아래와 같다.

In [1]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]

### 6.1.3 SGD의 단점

* SGD는 단순하고 구현도 쉽지만, 문제에 따라서는 비효율적일 때가 있다.
* 다음 함수의 최솟값을 구해보자.

## $f(x, y) = {1\over20}{x^2} + {y^2}$

* 그래프와 등고선을 그려보면 다음과 같다.<br/>
# <img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fcdt9de%2FbtqJ1MwR1Qs%2FT38qmey9lUxkAmk51UQcPk%2Fimg.png" align="left" width='500px' height='500px'>

* 기울기는 다음과 같다.<br/>
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FmNk3x%2FbtqJWRTsTgr%2FLQaFKm9g6MKAxTWMRxG1Ak%2Fimg.png" align="left" width='300px' height='300px'><br/>

* 이 기울기는 y축 방향은 크고 x축 방향은 작다는 것이 특징이다.
* 위 식이 최솟값이 되는 장소는 (x, y) = (0, 0) 이지만, 그림이 보여주는 기울기 대부분은 (0, 0)방향을 가리키지 않는다.
* 위 함수에 SGD를 적용하면 다음과 같다. <br/>
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FEZ0lW%2FbtqJXWmKdP6%2FCD23tT9hP6Y9lRxL4ksP0k%2Fimg.png" align="left" width='300px' height='300px'><br/>

* SGD는 상당히 비효율적인 움직임을 보여준다.
* SGD의 단점은 <span style='background-color:yellow'>방향에 따라 기울기가 달라지는 함수에서는 탐색 경로가 비효율적이다는 것이다.</span>
* SGD의 이러한 단점을 개선해주는 <span style='color:blue'>모멘텀, AdaGrad, Adam</span>이라는 세 방법을 알아보자.

### 6.1.4 모멘텀

* 모멘텀은 운동량을 뜻하는 단어로, 물리와 관계가 있다.
* 수식으로는 다음과 같다. <br/>
## $\mathbf{v} \leftarrow {\alpha \mathbf{v}} - {\eta{\partial{L}\over\partial{\mathbf{W}}}}$ <br/>
## $\mathbf{W} \leftarrow \mathbf{W} + \mathbf{v}$ <br/>
* SGD처럼 여기에서도 $\mathbf{W}$는 갱신할 가중치 매개변수, $\partial{L}\over\partial{\mathbf{W}}$는 $\mathbf{W}$에 대한 손실함수의 기울기, $\eta$는 학습률이다.
* $\mathbf{v}$라는 변수는 물리에서 말하는 속도에 해당한다.

In [2]:
class Momentum:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
        
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)
                
        for key in params.key():
            self.v[key] = self.momentum * self.v[key] - self.lr * grads[key]
            params[key] += self.v[key]

* 모멘텀에 의한 최적화 갱신 경로를 살펴보자.

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FpWahY%2FbtqJ1KMBFiI%2FSJ8P6njpgqqy7FXH2qKKjk%2Fimg.png" align="left" width='400px' height='400px'>

* SGD와 비교하면 지그재그 정도가 덜하다.
* 이는 $x$축의 힘은 아주 작지만 방향은 변하지 않아서 한 방향으로 일정하게 가속하기 때문이다.
* 거꾸로 $y$축의 힘은 크지만 위아래로 번갈아 받아서 상충하여 $y$출 방향의 속도는 안정적이지 않다.
* 전체적으로는 SGD보다 $x$축 방향으로 빠르게 다가가 지그재그 움직임이 줄어든다.

### 6.15 AdaGrad

* 신경망 학습에서는 학습률 값이 중요하다. 이 값이 너무 작으면 학습시간이 너무 길어지고, 반대로 너무 크면 발산하여 학습이 제대로 이뤄지지 않는다.
* 이 학습률을 정하는 효과적인 기술로 <span style='color:blue'>학습률 감소</span>가 있다.
* 이는 <span style='background-color:yellow'>학습을 진행하면서 학습률을 점차 줄여가는 방법이다.</span>
* 처음에는 크게 학습하다가 조금씩 작게 학습한다는 얘기로, 실제 신경망 학습에 자주 쓰인다.

* 학습률을 서서히 낮추는 가장 간단한 방법은 매개변수 전체의 학습률 값을 일괄적으로 낮추는 것이다.
* 이를 더욱 발전시킨 것이 <span style='color:blue'>AdaGrad</span>이다.
* AdaGrad는 '각각의' 매개변수에 '맞춤형' 값을 만들어준다.

* AdaGrad는 개별 매개변수에 적응적으로 학습률을 조정하면서 학습을 진행한다.
* AdaGrad 갱신 방법은 수식으로는 다음과 같다.

## $\mathbf{h} \leftarrow \mathbf{h} + {\partial L \over \partial W} \odot {\partial L \over \partial W}$ <br/>
## $\mathbf{W} \leftarrow \mathbf{W} - {\eta}{1 \over \sqrt{\mathbf{h}}}{\partial L \over \partial W}{\partial L \over \partial W}$

* 여기서는 $\mathbf{h}$라는 새로운 변수가 등장한다.
* $\mathbf{h}$는 기존 기울기의 값을 제곱하여 계속 더해준다.
* 그리고 매개변수를 갱신 할 때 $1 \over \sqrt{h}$을 곱해 학습률을 조정한다.
* 매개변수의 원소 중에서 많이 움직인(크게 갱신된) 원소는 학습률이 낮아진다는 뜻인데, 다시 말해 학습률 감소가 매개변수의 원소마다 다르게 적용됨을 뜻한다.

In [4]:
class  AdaGrad:
    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
                
        for key in params.key():
            self.h[key] += grads[key] * grads[key]
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)

* 여기서 주의할 것은 마지막 줄에서 '1e-7' 이라는 작은 값을 더하는 부분이다.
* 이 작은 값은 self.h[key]에 0이 담겨있다해도 0으로 나누는 사태를 방지해준다.
* AdaGrad를 사용해서 최적화 문제를 풀면 다음과 같다.

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FcHAg5e%2FbtqJWTcMctz%2F6foaiv3ZaaD7mrJ8ZvaRe0%2Fimg.png" align="left" width='400px' height='400px'>

* 그림을 보면 최솟값을 향해 효율적으로 움직이는 것을 알 수 있다.
* $y$축 방향은 기울기가 커서 처음에는 크게 움직이지만, 그 큰 움직임에 비례해 갱신 정도도 큰 폭으로 작아지도록 조정된다.
* 그래서 $y$축 방향으로 갱신 강도가 빠르게 약해지고, 지그재그 움직임이 줄어든다.

### 6.1.6 Adam

* Adam은 모멘텀과 AdaGrad 두 기법을 융합한 방법이다.
* 두 방법의 이점을 조합했으므로 매개변수 공간을 효율적으로 탐색해줄 것이라고 기대할 수 있다.
* 또, 하이퍼파라미터의 '편향 보정'이 진행된다는 점도 Adam의 특징이다.
* Adam을 사용하여 최적화 문제를 풀면 다음과 같다.

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fdt8EvM%2FbtqJWrm9RLv%2FkqUDwalmGKKWswlZ8waVoK%2Fimg.png" align="left" width='400px' height='400px'>

* 그림과 같이 Adam의 갱신 과정도 그릇 바닥을 구르듯 움직인다.
* 모멘텀과 비슷한 패턴인데, 모멘텀 때보다 좌우 흔들림이 적다.
* 이는 학습의 갱신 강도를 적응적으로  조정해서 얻는 혜택이다.

### 6.1.7 어느 갱신 방법을 이용할 것인가?

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FKZaLP%2FbtqJ4FYkYOb%2FaaCOENlEHCLE0z2i4h8fj1%2Fimg.png" align="left" width='700px' height='700px'>

* 그림과 같이 사용한 기법에 따라 갱신 경로가 다르다.
* 이 그림만 보면 AdaGrad가 가장 나아보이지만, 사실 그 결과는 풀어야 할 문제가 무엇이냐에 따라 달라지므로 주의해야 한다.
* 또, 당연하지만 하이퍼파라미터를 어떻게 설정하느냐에 따라서도 결과가 바뀐다.
* 각자의 장단이 있어 상황에 따라 시도를 해야 한다.

### 6.1.8 MNIST 데이터셋으로 본 갱신 방법 비교

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FctCKCE%2FbtqJXVOYCnA%2FiW4uvT1Zwl0kNaZbdLLGMk%2Fimg.png" align="left" width='500px' height='500px'>

* 학습 결과를 보면 SGD의 학습 진도가 제일 느리다.
* 나머지 세 기법의 진도는 비슷한데, 잘 보면 AdaGrad가 조금 더 빠른 것 같다.
* 이 실험에서 주의할 점은 하이퍼파라미터인 학습률과 신경망의 구조(층의 깊이 등)에 따라 결과가 달라진다는 것이다.